In [34]:
import pandas as pd

In [35]:
df = pd.read_excel('patient-medical-records-2024.xlsx')

In [36]:
df.columns

Index(['admissions.case_year', 'admissions.hash', 'admissions.id', 'exams.age',
       'exams.age_unit', 'exams.attitude', 'exams.bcs', 'exams.body',
       'exams.cardiopulmonary', 'exams.cns', 'exams.comments',
       'exams.dehydration', 'exams.examined_at', 'exams.examiner',
       'exams.forelimb', 'exams.gastrointestinal', 'exams.head',
       'exams.hindlimb', 'exams.integument', 'exams.mm_color',
       'exams.mm_texture', 'exams.musculoskeletal', 'exams.nutrition',
       'exams.sex', 'exams.temperature', 'exams.temperature_unit',
       'exams.treatment', 'exams.type', 'exams.weight', 'exams.weight_unit',
       'patient_locations.area', 'patient_locations.comments',
       'patient_locations.enclosure', 'patient_locations.moved_in_at',
       'patient_locations.where_holding', 'patients.address_found',
       'patients.admitted_at', 'patients.admitted_by', 'patients.band',
       'patients.carcass_saved', 'patients.care_by_rescuer',
       'patients.city_found', 'patients.cl

## Drop Rows where there is no geolocation information

In [37]:
df[['patients.address_found', 'patients.county_found', 'patients.city_found']]

,patients.address_found,patients.county_found,patients.city_found
0,1179 SR-712,Loudoun,Upperville
1,3250 Bust Head Rd,Fauquier,The Plains
2,Intersection of Greenway and Rt 7,Loudoun,Leesburg
3,109 Keverne Ct.,Frederick,Stephens City
4,21024 Fox Hollow Ln.,Loudoun,Leesburg
...,...,...,...
3312,225 Sister Chipmunk Ln,Frederick,Clearbrook
3313,20333 Tanager Place,Loudoun,Leesburg
3314,Found on a walking trail behind Dockside Terra...,Fairfax,Sterling
3315,297 Cedar Mtn. Ln.,Warren,Front Royal


Some of the data in 'patients.address_found' is not useful for programatically determining lat long coordinates. We want to drop the rows where the address_found starts with a alphabetic character AND either lat_found or lng_found is null

In [38]:
# determine which rows have the 'patients.address_found' column starting with a non-numeric character 
# df[df['patients.address_found'].str[0].str.isnumeric() == True]
# Set the 'patients.address_found' column to null for these rows
df.loc[df['patients.address_found'].str[0].str.isnumeric() == False, 'patients.address_found'] = None


In [39]:
# Drop rows where the following columns are empty: 'patients.address_found', 'patients.lat_found', 'patients.long_found'
df = df.dropna(subset=['patients.address_found', 'patients.lat_found', 'patients.lng_found'], how='all')

## Convert Address to Geolocation
For rows where we have an address but no lat or long info

In [8]:
df[['patients.address_found', 'patients.lat_found', 'patients.lng_found']]

,patients.address_found,patients.lat_found,patients.lng_found
0,1179 SR-712,NaN,NaN
1,3250 Bust Head Rd,NaN,NaN
2,Intersection of Greenway and Rt 7,NaN,NaN
3,109 Keverne Ct.,NaN,NaN
4,21024 Fox Hollow Ln.,NaN,NaN
...,...,...,...
3312,225 Sister Chipmunk Ln,NaN,NaN
3313,20333 Tanager Place,NaN,NaN
3314,Found on a walking trail behind Dockside Terra...,39.040332,-77.354295
3315,297 Cedar Mtn. Ln.,NaN,NaN


In [17]:
# Drop rows where either lat_found or _lng_found is NaN
df = df.dropna(subset=['patients.lat_found', 'patients.lng_found'])

In [ ]:
df[p[]]

In [21]:
df[['patients.lat_found', 'patients.lng_found']]

,patients.lat_found,patients.lng_found
48,38.831823,-7.752172e+01
75,39.034220,-7.740088e+01
174,39.501650,-7.801266e+01
183,38.841123,-7.792664e+07
186,39.039670,-7.791998e+07
...,...,...
3243,39.165549,-7.818914e+01
3265,39.030428,-7.750816e+01
3272,39.040514,-7.742277e+01
3300,39.086555,-7.813949e+01


## Determine Which ones are Vehicle Collision Related

In [13]:
df['patients.reasons_for_admission'].value_counts()

patients.reasons_for_admission
Cat attack                                       87
HBV                                              74
Abandoned                                        40
Dog attack                                       37
Unable to fly                                    36
                                                 ..
Found with legs malformed, unable to stand        1
Found in road, suspect HBV                        1
Newborn, found alone on driveway                  1
Found in middle of road, injured, suspect HBV     1
hanging in tree caught with fishing line          1
Name: count, Length: 2119, dtype: int64

In [14]:
vehicle_collision_strings = ['HBV', 'Found in road, suspect HBV']

In [15]:
df[df['patients.reasons_for_admission'].isin(vehicle_collision_strings)]

,admissions.case_year,admissions.hash,admissions.id,exams.age,exams.age_unit,exams.attitude,exams.bcs,exams.body,exams.cardiopulmonary,exams.cns,...,people.notes,people.organization,people.phone,people.postal_code,people.subdivision,species.class,species.family,species.genus,species.order,species.species
32,2024,NaN,33,NaN,Adult,Quiet,Plump,NaN,NaN,NaN,...,"brought VaOp to Judie Graham, Native Wildlife ...",Judie Graham,5.409032e+09,22553,VA,Mammalia,Didelphidae,Didelphis,Didelphimorphia,virginiana
267,2024,NaN,268,NaN,Adult,Stuporous,Thin,"L carapace completely crushed and gutted, drie...",NaN,NaN,...,NaN,Loudoun County Animal Services,7.037770e+09,20175,VA,Reptilia,Emydidae,Chrysemys,Testudines,Picta
1052,2024,NaN,1052,NaN,Adult,Depressed,Reasonable,"multiple cranial carapace fractures, bridge fr...",NaN,possible skull trauma,...,NaN,NaN,5.402778e+09,22620,VA,Reptilia,Emydidae,Terrapene,Testudines,Carolina
1080,2024,NaN,1080,NaN,Adult,Quiet,Reasonable,severe cranial carapace fx from midline to lef...,NaN,NaN,...,NaN,NaN,5.404652e+09,NaN,VA,Reptilia,Emydidae,Chrysemys,Testudines,Picta
1102,2024,NaN,1102,NaN,Adult,Alert,Reasonable,NaN,NaN,NaN,...,Fairfax Co APP brought to Pender.,Pender Exotics Veterinary Center,NaN,NaN,VA,Reptilia,Chelydridae,Chelydra,Testudines,Serpentina
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,2024,NaN,3182,NaN,Sub-adult,Quiet,Reasonable,NaN,NaN,NaN,...,NaN,NaN,3.047036e+09,26704,WV,Reptilia,Emydidae,Terrapene,Testudines,Carolina
3196,2024,NaN,3195,NaN,Adult,Quiet,Good,L shaped cr carapace fx directly over R should...,NaN,NaN,...,NaN,NaN,6.812713e+09,26865,WV,Reptilia,Emydidae,Terrapene,Testudines,Carolina
3259,2024,NaN,3258,NaN,Adult,Stuporous,Good,"severe cranial carapace and skull/jaw fxs, nea...",NaN,NaN,...,NaN,NaN,NaN,NaN,VA,Reptilia,Chelydridae,Chelydra,Testudines,Serpentina
3280,2024,NaN,3279,NaN,Adult,Obtunded,Reasonable,fully crushed cranial carapace w/ complete sev...,NaN,NaN,...,NaN,NaN,7.039661e+09,22611,VA,Reptilia,Emydidae,Terrapene,Testudines,Carolina
